#### Importando Bibliotecas

In [ ]:
import duckdb
import os

from dotenv import load_dotenv

#### Primeiros Passos com MotherDuck

##### Configurando Token

In [ ]:
dotenv_path = "../config/.env"
load_dotenv(dotenv_path)

motherduck_token =  os.getenv("motherduck_token")

##### Inicializando conexão do DuckDB ao MotherDuck

In [ ]:
con = duckdb.connect(f"md:?motherduck_token={motherduck_token}")

In [ ]:
con.sql("SHOW DATABASES")

##### Lendo o CSV

In [ ]:
# usando AUTO_DETECT=TRUE para inferir o schema automático
con.sql("SELECT * FROM read_csv('../data/input/contratos_202404.csv', AUTO_DETECT=TRUE) LIMIT 5")

##### Inserindo o CSV em uma tabela temporária chamada "stg"

In [ ]:
con.sql("CREATE OR REPLACE TEMP TABLE stg AS SELECT * FROM read_csv('../data/input/contratos_202404.csv', AUTO_DETECT=TRUE)")

In [ ]:
con.sql("SELECT * FROM temp.stg")

##### Salvando os dados no MotherDuck

In [ ]:
# criando o banco de dados
con.sql("CREATE DATABASE data_processing")

In [ ]:
# criando um schema
con.sql("CREATE SCHEMA data_processing.dbo")

In [ ]:
# criando a tabela automaticamente e inserindo os dados
# assim como a stg, estamos inferindo o schema da tabela automaticamente
con.sql("CREATE TABLE data_processing.dbo.contratos AS SELECT * FROM temp.stg")

##### Contando as Linhas

In [ ]:
con.sql("SELECT COUNT(*) FROM data_processing.dbo.contratos")

##### Inserindo mais dados

In [ ]:
# inserindo a stg novamente
con.sql("INSERT INTO data_processing.dbo.contratos SELECT * FROM temp.stg")

In [ ]:
# contando novamente
con.sql("SELECT COUNT(*) FROM data_processing.dbo.contratos")

##### Criando uma View

In [ ]:
con.sql("CREATE VIEW data_processing.dbo.view_rescisao AS SELECT * FROM data_processing.dbo.contratos WHERE DataRescisao IS NOT NULL")

##### Selecionando a View

In [ ]:
con.sql("SELECT * FROM data_processing.dbo.view_rescisao")

##### Usando CTE

In [ ]:
con.sql("WITH cteTeste AS (SELECT * FROM data_processing.dbo.view_rescisao) SELECT * FROM cteTeste WHERE DataAssinatura = '2023-08-25'")

#### Continua.....